In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [ ]:
#TODO
#1. when all teams inserted rank = index
#2. make it more sensitive to blowouts
#3. take care of zeros
#4. figure 0-0
#5. not enough x's
#6. injuries
#7. 5 xs 10 1s 10 2s, this is against the x
#8. team value
#9. stability - points per game std V
#10. international contests
#11. red cards
#12. days from
#13. points as a column in history

In [2]:
def to_csv (dataframe, name_without_dot_csv, mahzor):
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True);
    path = '/content/drive/My Drive/Colab Notebooks/histories ' + str(mahzor) + "/" + name_without_dot_csv +'.csv'
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      dataframe.to_csv(f)

In [3]:
def calc_points_one_game (index_home, index_away, home_team_goals, away_team_goals, days_home, days_away, updated_table, values):
  home_team_rank, away_team_rank = updated_table [index_home], updated_table [index_away]
  team_goal_counts = (1 - away_team_rank/30)*((home_team_goals) ** 1.2) * (1 - (days_home-3)/32)
  rival_goal_counts = (1 - (20 - away_team_rank)/30)*(away_team_goals ** 1.2) * (1 - (4-(days_away-3))/32)
  egd = team_goal_counts - rival_goal_counts #effective goal differential
  ph = np.abs (egd-0.15) ** (1+(home_team_goals**1.2 - away_team_goals**1.2)/20) * np.sign (egd-0.15)
  ph += compare_to_last_year (index_home, index_away, updated_table) + np.log(values[index_away]) / 8  #points home team
  pa = (-1) * (np.abs (egd)** (1-(home_team_goals**1.2 - away_team_goals**1.2)/20) * np.sign (egd)) #points away team
  pa += compare_to_last_year (index_away, index_home, updated_table) + np.log(values[index_home]) / 8
  return ph, pa

In [4]:
def points_calc (histories, index, updated_table, absence, values): #TODO
  history = histories [index]
  team_goal_counts = (1 -
                       history.loc [:, "Vs"]/30) * (history.loc [:, "team goals"] + absence) ** 1.2 * (1 -
                        (history.loc [:, "Days from last game"]-3)/32)
  rival_goal_counts = (1 -
   (20 - history.loc [:, "Vs"])/30) * (history.loc [:, "rival goals"] ** 1.2) *(1 - (4-(history.loc [:, "Days from last game"]-3))/32)
  egd = team_goal_counts - rival_goal_counts
  points = np.sign (egd) * (np.abs (egd)**(1 + ((history.loc [:, "team goals"] ** 1.2) - (history.loc [:, "rival goals"] ** 1.2))/20))
  points = points + (points == 0) * (updated_table [index] - history.loc [:, "Vs"])/20 + (1 - history.loc [:, "Home game?"])*0.15
  for j in range (len (points)):
    points [j] += compare_to_last_year (index, history.loc [j, "Vs"], updated_table) + np.log(values[index]) / 8
  return points

In [5]:
def eppg (histories, index, updated_table, absence, values): #effective points per game
  points = points_calc (histories, index, updated_table, absence, values)
  #return (np.mean(points [-3:])*0.75 + np.mean(points)/4)
  return (points [len (points) -1] / 3 + np.mean(points [-3:]) / 3 + np.mean(points)/3)

In [6]:
def compare_to_last_year (team_index, rival_index, updated_table):
  last_year_standings = ["", 2, 10, 1, 3, 8, 4, 6, 11, 16, 22, 15,
                       7, 14, 12, 9, 5, 23, 13, 21, 17] #number 20 (almeria) was placed 17th
  distance_now = updated_table [team_index] - updated_table [rival_index]
  distance_from_yourself_last_year = updated_table [team_index] - last_year_standings [team_index]
  v = distance_now * 1.05 ** (distance_from_yourself_last_year) / 20
  if distance_from_yourself_last_year == 0:
    return v
  if v / distance_from_yourself_last_year > 0:
    return v
  return -v

In [7]:
teams = ["", 'real madrid', 'girona', 'barcelona', 'atletico', 'bilbao', 'sociedad', 'betis', 'rayo', 'valencia',
              "las palmas", "getafe", "osasuna", "cadiz", "sevilla", "mallorca", "villareal", "alaves", "celta",
              "granada", "almeria"]
last_year_standings = ["", 2, 10, 1, 3, 8, 4, 6, 11, 16, 22, 15,
                       7, 14, 12, 9, 5, 23, 13, 21, 17] #number 20 (almeria) was placed 17th
values = ["", 44.61, 6.44, 41.05, 19.67, 8.75, 16.64, 7.8, 3.48, 7.73, 2.07, 4.73, 4.86, 2.13, 6.93,
          4.03, 7.76, 2.27, 4.91, 2.84, 3.80]

In [8]:
absences = ["", 2, 0.4, 1.5, 2, 0.5, 0, 0.3, 0, 0.25, 0.2,
            0.7, 0, 0, 0, 0.2, 0.25, 0.25, 0.5, 0, 0]

In [32]:
updated_table = ["", 1, 2, 4, 3, 5, 6, 7, 10, 9, 8, 11, 14, 16,
                 15, 17, 17, 12, 13, 19, 20] #updated table [1] is real madrid's spot
# updated_table = [j for j in range (21)]

In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True);
histories = [""]
for j in range (1, 21):
  histories.append(pd.read_csv ("/content/drive/MyDrive/Colab Notebooks/histories 13/" + teams [j] + " history mahzor 13.csv").drop(['Unnamed: 0'], axis = 1))
histories [15].loc [1, "Vs"] = 16

Mounted at /content/drive


In [33]:
instability = np.array ([np.std (points_calc (histories, j, updated_table, absences [j], values)) for j in range (1, 21)])
#instability = np.sqrt (np.array (instability))

In [ ]:
#absences = [betis 0.3 osasuna 0 girona 0 celta 1/2] cadiz 0 sevilla 0 rayo 0 rsoc 0 bilbao 1/2
# val 1/4  alaves 1/4 vllr 1/4 granada 0 real 2 atl 2 las palmas 0.2 mallorca 0.2 bar 1.5 alm 0 get 0.7

In [34]:
days_froms = [0, 15, 16, 13, 14, 17, 15, 14, 14, 14, 15, 14, 15, 3, 14, 4, 14, 12, 15, 13, 14]

In [ ]:

Las Palmas Getafe 5 6

Girona Valencia 5 7

Bilbao Vallecano 5 7

Rma gra 6 7

Osa soc  7 7

Mll alaves 4 9

Almeria betis 8 7

Sev vll 7 7

Bar atl 8 8

Celta cadiz 9 5



In [52]:
home_away = (18, 13)
home_days_away_days = 9, 5
days_froms [home_away [0]] = home_days_away_days[0]
days_froms [home_away [1]] = home_days_away_days[1]
home_absence, away_absence = absences [home_away [0]], absences [home_away [1]] #ABSENCE IS HELPING OTHER TEAM RATHER THAN HURTING YOU
a, b = eppg (histories, home_away[0], updated_table, away_absence, values), eppg (histories, home_away[1], updated_table, home_absence, values)
df = pd.DataFrame()
result = []
for hg in range (5):
  for ag in range (5):
    result.append (str (hg) + "-" + str (ag))
    (c, d) = calc_points_one_game (home_away[0], home_away[1], hg, ag,
                                   home_days_away_days[0], home_days_away_days[1], updated_table, values)
    tmp = pd.DataFrame ({"result": [str (hg) + "-" + str (ag)], "points home": [c], "points away": [d]})
    df = pd.concat ([df, tmp])
pha, paa = np.array(df["points home"]-a), np.array(df["points away"]-b)
ins_home, ins_away = instability [home_away [0] - 1], instability [home_away [1] - 1]
weighted = ins_away * pha**2 + ins_home * paa**2
j = np.argmin(weighted)
df ["pha**2 + paa**2"] = pha**2 + paa**2
#df ["pha**2 + paa**2 weighted"] = weighted

odds_array = 1 / weighted
print (result [j], str (np.round (odds_array [j] / (sum (odds_array)), 2)))
odds_x = (odds_array [0] + odds_array [6] + odds_array [12] + odds_array [18] + odds_array [24]) / (sum (odds_array))
odds_1 = (odds_array [5] + odds_array [10] + odds_array [11] + odds_array [15] + odds_array [16] +
          odds_array [17] + odds_array [20] + odds_array [21] + odds_array [22] + odds_array [23]) / (sum (odds_array))
# odds_x = (odds_array [0] + odds_array [5] + odds_array [10] + odds_array [15]) / (sum (odds_array))
# odds_1 = (odds_array [4] + odds_array [8] + odds_array [9] + odds_array [12] + odds_array [13] +
#           odds_array [14]) / (sum (odds_array))
# odds_1_0 = odds_array [5] / sum(odds_array)
df ['odd'] = odds_array / sum (odds_array)
cols = df.columns.tolist()
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df = df[cols]
df = df.sort_values(by=['odd'], ascending=False)
odds_2 = 1 - odds_x - odds_1
print (np.round (odds_1, 2), np.round (odds_x, 2), np.round (1 - np.round (odds_1, 2) - np.round (odds_x, 2), 2))
df.head(3)

1-0 0.27
0.83 0.14 0.03


,result,odd,points home,points away,pha**2 + paa**2
0,1-0,0.265208,0.157407,-0.033721,0.123468
0,4-2,0.184935,-0.063741,0.182776,0.202725
0,2-1,0.138427,-0.133749,0.293848,0.270904


In [42]:
df.head(3)

,result,odd,points home,points away,pha**2 + paa**2
0,4-4,0.160853,-0.010300,-0.136877,0.261280
0,3-3,0.150828,-0.045611,-0.101566,0.278755
0,2-2,0.140379,-0.078606,-0.068571,0.299592


In [ ]:
j = 0
vss = []
for history in histories [1:]:
  j+= 1
  if j in vss:
    continue
  print (teams [j])
  vs = input ('vs ')
  if vs == "dnp":
    continue
  vss.append (int (vs))
  home = input ('home ')
  goals = input ('goals ')
  rival_goalss = input ('rival_goals ')
  days_from = days_froms [j]
  history.loc [len (history)] = [int (vs), int (home), int (goals), int (rival_goalss), int (days_from)]
  days_2 = days_froms [int (vs)]
  histories [int (vs)].loc [len (histories [int (vs)])] = [j, np.abs (int (home)-1), int (rival_goalss), int (goals), int (days_from)]

In [38]:
mahzor = 14
for j, history in enumerate (histories):
  if not j:
    continue
  to_csv (history, teams [j] + " history mahzor " + str (mahzor), mahzor);

Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
Mounted at /content/drive
